# Modules To Import

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas

# Crawler Class
> #### Note On Season Identifiers
> Season years are represented in th from of "Season year-year+1" (e.g. Season 2021-2022). To have less data complexity and more efficieny I choose "year+1" from each season in the form of "year-year+1" as a key for that specific season because the website [basketball-reference.com](www.basketball-reference.com) is also doing the same.
> 
>  **TL;DR**: The key "2024" points to the 2023-24 season.  

In [51]:
class Crawler:
    def get_mvps(self, mvp_page):
        mvp_soup = BeautifulSoup(mvp_page.content, "html.parser")
        mvp_soup = mvp_soup.find('table', id="mvp_NBA").tbody.find_all("tr")
        return mvp_soup

In [24]:
mvp_page = requests.get("https://www.basketball-reference.com/awards/mvp.html")

In [52]:
c = Crawler()
mvp = c.get_mvps(mvp_page)

print(mvp)

[<tr><th class="left" data-stat="season" scope="row"><a href="/leagues/NBA_2024.html">2023-24</a></th><td class="left" data-stat="lg_id"><a href="/leagues/NBA_2024.html">NBA</a></td><td class="left" csk="Jokić,Nikola" data-append-csv="jokicni01" data-stat="player"><a href="/players/j/jokicni01.html">Nikola Jokić</a></td><td class="center" data-stat="voting"> (<a href="/awards/awards_2024.html#mvp">V</a>)</td><td class="right" data-stat="age">28</td><td class="left" data-stat="team_id"><a href="/teams/DEN/2024.html">DEN</a></td><td class="right" data-stat="g">79</td><td class="right" data-stat="mp_per_g">34.6</td><td class="right" data-stat="pts_per_g">26.4</td><td class="right" data-stat="trb_per_g">12.4</td><td class="right" data-stat="ast_per_g">9.0</td><td class="right" data-stat="stl_per_g">1.4</td><td class="right" data-stat="blk_per_g">0.9</td><td class="right" data-stat="fg_pct">.583</td><td class="right" data-stat="fg3_pct">.359</td><td class="right" data-stat="ft_pct">.817</td